In [ ]:
!pip install scikit-surprise
!pip install optuna

     |████████████████████████████████| 11.8MB 294kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617671 sha256=f6eba63dd6b2cc40c7f10704d9447cbdd296c1995251350d379b72bf940b2b72
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
     |████████████████████████████████| 307kB 4.0MB/s 
     |████████████████████████████████| 174kB 22.0MB/s 
     |████████████████████████████████| 81kB 7.9MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 143kB 18.6MB/s 
     |████████████████████████████████| 112kB 18.7MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11136 sha256=3ddb6ea0aa01262c8ce4a95f1f7590dd18de138316354dadba2e4a1dd6a3cb82
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2

In [ ]:
# Mount Google Drive and set data paths.
import os
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive')
DATA_PATH = "/content/gdrive/My Drive/ETH/Computational Intelligence Lab/CIL-Project/data"
TRAIN_DATA_PATH = os.path.join(DATA_PATH, "data_train.csv")
TEST_DATA_PATH = os.path.join(DATA_PATH, "data_test.csv")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import surprise
from surprise import accuracy
from surprise.model_selection import cross_validate
import optuna

In [ ]:
def load_data(file_path: str, full_dataset: bool, train_val_split: bool, random_seed: int = 0, train_size: float = 0):
    data_pd = pd.read_csv(file_path)

    # Reduce Dataset for Testing
    if not full_dataset:
        data_pd = data_pd.head(10000)

    if train_val_split:
        train_pd, val_pd = train_test_split(data_pd, train_size=train_size, random_state=random_seed)
        return train_pd, val_pd
    else:
        return data_pd

def __extract_users_items_ratings(data_pd: pd.DataFrame):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    ratings = data_pd.Prediction.values
    return users, movies, ratings

def create_surprise_data(data_pd):
    users, movies, ratings = __extract_users_items_ratings(data_pd)

    df = pd.DataFrame({
        'users': users,
        'movies': movies,
        'ratings': ratings
    })
    reader = surprise.Reader(rating_scale=(1, 5))
    return surprise.Dataset.load_from_df(df[['users', 'movies', 'ratings']], reader=reader)

def __get_tensors_from_dataframe(data_pd: pd.DataFrame):
    users, movies, ratings = __extract_users_items_ratings(data_pd)
    users_torch = torch.tensor(users, dtype=torch.int64)
    movies_torch = torch.tensor(movies, dtype=torch.int64)
    ratings_torch = torch.tensor(ratings, dtype=torch.int64)

    return users_torch, movies_torch, ratings_torch


def create_dataset(data_pd: pd.DataFrame, test_dataset: bool = False):
    users_torch, movies_torch, ratings_torch = __get_tensors_from_dataframe(data_pd)

    if not test_dataset:
        return TensorDataset(users_torch, movies_torch, ratings_torch)
    else:
        test_ids = data_pd.Id
        return test_ids, TensorDataset(users_torch, movies_torch)

In [ ]:
random_seed = 42
full_dataset = True

np.random.seed(random_seed)

train_pd = load_data(
    file_path=TRAIN_DATA_PATH,
    full_dataset=full_dataset,
    train_val_split=False
)

In [ ]:
train_data = create_surprise_data(train_pd)
trainset, testset = surprise.model_selection.train_test_split(train_data, test_size=0.2)

In [ ]:
from surprise import NMF

def objective(trial):
    biased = trial.suggest_categorical("biased", [False, True])
    n_factors = trial.suggest_int("n_factors", 2, 100)
    n_epochs = trial.suggest_int("n_epochs", 10, 100)
    init_low = trial.suggest_int("init_low", 0, 5)
    init_high = trial.suggest_int("init_high", 0 , 5)

    algo = NMF(biased=biased, n_factors=n_factors, n_epochs=n_epochs, init_low=init_low, init_high=init_high)
    
    algo.fit(trainset)
    predictions = algo.test(testset)
    rmse = accuracy.rmse(predictions)
    return rmse

In [ ]:
study = optuna.create_study(direction="minimize")

[I 2021-06-08 19:04:01,053] A new study created in memory with name: no-name-4a5c1cdc-844c-46e1-80e6-95c7f4065b17


In [ ]:
# Enqueue trials with default parameters.
study.enqueue_trial(
    {
        "biased": False,
        "n_factors": 15,
        "n_epochs": 50,
        "init_low": 0,
        "init_high": 1
    }
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: ExperimentalWarning:

enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study.py:858: ExperimentalWarning:

create_trial is experimental (supported from v2.0.0). The interface can change in the future.

/usr/local/lib/python3.7/dist-packages/optuna/study.py:858: ExperimentalWarning:

add_trial is experimental (supported from v2.0.0). The interface can change in the future.



In [ ]:
study.optimize(objective, n_trials=80)

[I 2021-06-08 19:05:11,125] Trial 0 finished with value: 1.0090560166643434 and parameters: {'biased': False, 'n_factors': 15, 'n_epochs': 50, 'init_low': 0, 'init_high': 1}. Best is trial 0 with value: 1.0090560166643434.


RMSE: 1.0091


[I 2021-06-08 19:08:13,184] Trial 1 finished with value: 1.0036458570903077 and parameters: {'biased': True, 'n_factors': 33, 'n_epochs': 98, 'init_low': 4, 'init_high': 4}. Best is trial 1 with value: 1.0036458570903077.


RMSE: 1.0036


[I 2021-06-08 19:10:03,895] Trial 2 finished with value: 1.0814862694986065 and parameters: {'biased': True, 'n_factors': 85, 'n_epochs': 33, 'init_low': 0, 'init_high': 4}. Best is trial 1 with value: 1.0036458570903077.


...

RMSE: 1.0033


[I 2021-06-08 21:37:09,267] Trial 78 finished with value: 1.0027021843415784 and parameters: {'biased': True, 'n_factors': 25, 'n_epochs': 65, 'init_low': 2, 'init_high': 2}. Best is trial 26 with value: 1.0011956014445267.


RMSE: 1.0027


[I 2021-06-08 21:39:13,274] Trial 79 finished with value: 1.0034867543017707 and parameters: {'biased': True, 'n_factors': 34, 'n_epochs': 65, 'init_low': 2, 'init_high': 2}. Best is trial 26 with value: 1.0011956014445267.


RMSE: 1.0035


In [ ]:
# NOTE: We do not prune any trials since surprise does not support partial fit.
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  80
  Number of pruned trials:  0
  Number of complete trials:  80
Best trial:
  Value:  1.0011956014445267
  Params: 
    biased: True
    n_factors: 11
    n_epochs: 76
    init_low: 0
    init_high: 1


In [ ]:
test_pd = load_data(
    file_path=TEST_DATA_PATH,
    full_dataset=full_dataset,
    train_val_split=False
)

train_data = create_surprise_data(train_pd).build_full_trainset()
test_ids, test_data = create_dataset(test_pd, test_dataset=True)
test_ids = test_ids.to_numpy()

from surprise import NMF

biased = True
n_factors = 11
n_epochs = 76
init_low = 0
init_high = 1

algo = NMF(biased=biased, n_factors=n_factors, n_epochs=n_epochs, init_low=init_low, init_high=init_high)

algo.fit(train_data)

predictions = []
for user, movie in test_data:
    prediction = algo.predict(user.item(), movie.item()).est
    predictions.append(prediction)

output = np.stack((test_ids, predictions), axis=1)

pd.DataFrame(output, columns=["Id", "Prediction"]).to_csv("nmf_output.csv", index=None)
files.download("nmf_output.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>